In [25]:
import os
import numpy as np
from datetime import datetime
from tqdm import tqdm
from copy import deepcopy
from utils import multi_exp_data_to_fig

In [57]:
# hypothesis = 'pesto_protein_scores_ablation'
hypothesis = 'optimizer_batchsize_lr_gridsearch'
evaluation_type = 'validation'  # can be train / validation / test

In [58]:
experiments = os.listdir(f'../configurations/data/{hypothesis}')
experiments = [e.split('.json')[0] for e in experiments]

In [59]:
run_paths = []
for experiment in experiments:
    experiment_runs = os.listdir(f'../results/patch_to_score/hypotheses/{hypothesis}/{experiment}')
    try:
        experiment_runs = sorted(experiment_runs, key=lambda x: datetime.strptime(x.split('_')[0], '%Y-%m-%d'), reverse=True)
    except Exception as e:
        print(e)
    experiment_valid_run_found = False
    for updated_experiment_run in experiment_runs:
        run_path = f'{hypothesis}/{experiment}/{updated_experiment_run}'
        if 'fold_4' in os.listdir(f'../results/patch_to_score/hypotheses/{run_path}'):
            run_paths.append(run_path)
            experiment_valid_run_found = True
            break
    if not experiment_valid_run_found:
        print(f'experiment: {experiment} didnt finish, skipping')
len(run_paths)

experiment: batch_128-optimizer_adamw-lr_0.001 didnt finish, skipping
experiment: batch_128-optimizer_adam-lr_0.0001 didnt finish, skipping
experiment: batch_128-optimizer_adamw-lr_0.0001 didnt finish, skipping
experiment: batch_256-optimizer_adamw-lr_0.01 didnt finish, skipping
experiment: batch_256-optimizer_adamw-lr_0.001 didnt finish, skipping


31

In [60]:
base_paths = [f'../results/patch_to_score/hypotheses/{path}' for path in run_paths]

In [61]:
multi_exp_data = dict()
for base_path in tqdm(base_paths):
    exp_data = []
    exp_name = base_path.split('/')[-2]
    for fold in range(5):
        try:
            fold_labels = np.load(os.path.join(base_path, f'fold_{fold}', evaluation_type, 'labels.npy'))
            fold_predictions = np.load(os.path.join(base_path, f'fold_{fold}', evaluation_type, 'predictions.npy'))[:, 0]
            exp_data.append(np.stack([fold_labels, fold_predictions]))
        except Exception as e:
            pass
    multi_exp_data[exp_name] = exp_data

100%|██████████| 31/31 [00:02<00:00, 11.21it/s]


In [62]:
filtered_multi_exp_data = deepcopy(multi_exp_data)
filtered_multi_exp_data = {exp_name: exp_data for exp_name, exp_data in filtered_multi_exp_data.items() if ('sgd' not in exp_name and '0.0001' not in exp_name)}

In [63]:
fig = multi_exp_data_to_fig(filtered_multi_exp_data, 
                            show_std_in_legend=True, log_scale=True, resolution=1_000)
fig.show()

computing rocs


100%|██████████| 13/13 [00:00<00:00, 43.96it/s]


adding random roc
fig update
